In [32]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [2]:
# gonna try to make sure everything's good for DPO udpates in code, since it's a bit to run iteratively

In [3]:
tok = AutoTokenizer.from_pretrained("../outputs/models/bagofwords/bowtiny_dpo/")
mod = AutoModelForCausalLM.from_pretrained("../outputs/models/bagofwords/bowtiny_dpo/", device_map=0, torch_dtype=torch.bfloat16)

In [4]:
mod.eval()

OPTForCausalLM(
  (model): OPTModel(
    (decoder): OPTDecoder(
      (embed_tokens): Embedding(50272, 768, padding_idx=1)
      (embed_positions): OPTLearnedPositionalEmbedding(2050, 768)
      (final_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (layers): ModuleList(
        (0-11): 12 x OPTDecoderLayer(
          (self_attn): OPTAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (activation_fn): ReLU()
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), ep

In [35]:
def get_batch_logps(
    logits: torch.FloatTensor,
    labels: torch.LongTensor,
    average_log_prob: bool = False,
    label_pad_token_id: int = -100,
    is_encoder_decoder: bool = False,
) -> torch.FloatTensor:

    if logits.shape[:-1] != labels.shape:
        raise ValueError("Logits (batch and sequence length dim) and labels must have the same shape.")

    if not is_encoder_decoder:
        labels = labels[:, 1:].clone()
        logits = logits[:, :-1, :]
    loss_mask = labels != label_pad_token_id

    # dummy token; we'll ignore the losses on these tokens later
    labels[labels == label_pad_token_id] = 0

    per_token_logps = torch.gather(logits.log_softmax(-1), dim=2, index=labels.unsqueeze(2)).squeeze(2)

    if average_log_prob:
        return (per_token_logps * loss_mask).sum(-1) / loss_mask.sum(-1)
    else:
        return (per_token_logps * loss_mask).sum(-1)

In [64]:
inpstr = ["Newcastle won by 0-3", "Newcastle won by 1-1"]
inps = tok(inpstr, return_tensors="pt", padding=True).to(mod.device)

In [65]:
inps.input_ids

tensor([[    2,  4030, 24773,   351,    30,   321,    12,   246],
        [    2,  4030, 24773,   351,    30,   112,    12,   288]],
       device='cuda:0')

In [67]:
outs = mod(**inps)
get_batch_logps(outs.logits, inps.input_ids, average_log_prob=False, label_pad_token_id=tok.pad_token_id)

tensor([-35.2500, -31.6250], device='cuda:0', dtype=torch.bfloat16,
       grad_fn=<SumBackward1>)

In [29]:
outs.logits.shape

torch.Size([2, 12, 50272])